#### 모듈 / 라이브러리 불러오기 

In [3]:
#!pip install nltk
#!pip install spacy
#!pip install stanza
import stanza
import pandas as pd
import os
import re
import pickle
import sys 
import spacy 
import numpy as np
from IPython.display import clear_output 
from tqdm import tqdm 
from nltk import sent_tokenize
nlp = stanza.Pipeline(lang='en', processors='tokenize, pos, lemma')
clear_output()

In [5]:
# 새로운 resouce에 대하여 각 테이블 업데이트 
from Update_New_Tables import update_resource_stories, update_sentence_ids

# upos:lemma 입력 시 어떤 story/sentence 에서 등장 했는지 바로 보여 줌
from CountWords import create_merged_df, get_sentence_story_textbook

# Reference 단어를 제외한 upos:lemma table을 생성 & sr 및 st count 계산 
from ReferenceExclusion import get_sr_st_count, reference_words_exclusion 

# upos:lemma 입력 시 등장 sentence 를 테이블로 보여 줌 
# 전체 upos:lemma에 대하여 rs code / sr code 를 한 칼럼에 comma 로 구분한 테이블을 생성
from LemmaCount import get_ul_sentence, get_resource_code, get_story_code

clear_output()

#### 기존 파일 확인

In [6]:
with open('./data/resource_table_new.pickle', 'rb') as f:
    textbook_df = pickle.load(f)
with open('./data/story_table_text_new.pickle', 'rb') as f:
    story_df = pickle.load(f)
with open('./data/sentence_list_new.pickle', 'rb') as f:
    sentence_df = pickle.load(f)   
    
with open('./data/STID_XTID_new.pickle', 'rb') as f :
    STID_XTID=pickle.load(f)        
with open('./data/STID_ULID_new.pickle', 'rb') as f :
    STID_ULID=pickle.load(f) 

    
    
with open('./data/XTID.pickle', 'rb') as f :
    token_xpos=pickle.load(f)
with open('./data/ULID.pickle', 'rb') as f :
    lemma_upos=pickle.load(f)

    
    
with open('./data/entry_table_1208.pickle', 'rb') as f :
    entry_df=pickle.load(f)
with open('./data/entry_table_1211.pickle', 'rb') as f :
    entry_df_new=pickle.load(f)
    
#어휘 통합 파일 + entry table 통합 -> ET ID 부여 및 reference 포함 여부를 칼럼으로 갖고 있는 테이블 
with open('./data/vocab_reference_integrated.pickle', 'rb') as f :
    reference_intg=pickle.load(f)

In [18]:
new_ref=reference_intg[(reference_intg['Poly8K']=="X")&
               (reference_intg['FW']=="X")&
               (reference_intg['교육부3K']=="X")&
               (reference_intg['Longman3K']=="X")&
               (reference_intg['VOA1.5K']=="X")].reset_index(drop=True)
new_ref

,ENTRY,ET ID,FW,Poly8K,교육부3K,CEFR10K,LEXILE6K,Longman3K,VOA1.5K,WW 3K,VW 3K
0,rainwater,ET00060,X,X,X,X,3rd,X,X,X,X
1,purify,ET00061,X,X,X,X,X,X,X,X,Gr.3
2,agricultural,ET00080,X,X,X,X,4th,X,X,X,X
3,drying,ET00091,X,X,X,X,X,X,X,X,X
4,greenhouse,ET00129,X,X,X,X,4th,X,X,X,X
...,...,...,...,...,...,...,...,...,...,...,...
12269,zoomed,ET19884,X,X,X,X,X,X,X,K,X
12270,zooplankton,ET19885,X,X,X,X,9th to 12th,X,X,X,X
12271,zucchini,ET19886,X,X,X,B2,X,X,X,X,X
12272,zulu,ET19887,X,X,X,X,9th to 12th,X,X,X,X


In [20]:
new_ref[new_ref['LEXILE6K']=='6th to 8th']

,ENTRY,ET ID,FW,Poly8K,교육부3K,CEFR10K,LEXILE6K,Longman3K,VOA1.5K,WW 3K,VW 3K
68,businesspeople,ET00880,X,X,X,X,6th to 8th,X,X,X,X
96,porcupine,ET01105,X,X,X,X,6th to 8th,X,X,X,X
107,seafloor,ET01170,X,X,X,X,6th to 8th,X,X,X,X
120,wartime,ET01233,X,X,X,X,6th to 8th,X,X,X,X
129,gully,ET01304,X,X,X,X,6th to 8th,X,X,X,X
...,...,...,...,...,...,...,...,...,...,...,...
12239,workday,ET19833,X,X,X,X,6th to 8th,X,X,X,X
12249,writhe,ET19849,X,X,X,X,6th to 8th,X,X,X,Gr.6
12264,zealot,ET19876,X,X,X,X,6th to 8th,X,X,X,Gr.13
12267,ziggurat,ET19880,X,X,X,X,6th to 8th,X,X,X,X


In [72]:
lemma_upos
#[STID_ULID['ST ID']=='ST55833']

,UL ID,UPOS:Lemma
0,UL00001,NOUN:engineer
1,UL00002,VERB:like
2,UL00003,VERB:solve
3,UL00004,NOUN:problem
4,UL00005,VERB:fix
...,...,...
15076,UL15085,NOUN:pih
15077,UL15086,NOUN:toon
15078,UL15087,NOUN:urn
15079,UL15088,NOUN:482°c


# ------------------------------------------------------

In [68]:
elelist=os.listdir('/Users/jamielee/Library/CloudStorage/Dropbox/AIEI - 코퍼스/코퍼스 리소스/리소스 TXT/TXT_ELE')
elelist.remove('.DS_Store')
elelist = sorted(elelist)
new_ele_list=elelist[11:]
new_ele_list

['ENG-PTF-4.2-4-Waverider',
 'ENG-PTF-5.1-5-Aviator',
 'ENG-PTF-5.2-5-Rocketeer',
 'ENG-WDS-1.1-1-Rise and Shine',
 'ENG-WDS-1.2-1-Fly High',
 'ENG-WDS-2.1-2-All Aboard',
 'ENG-WDS-2.2-2-Set Sail',
 'ENG-WDS-3.1-3-Into the Horizon',
 'NFS-ERK-1-1-Eureka',
 'NFS-ERK-2-2-Eureka',
 'NFS-ERK-3-3-Eureka']

In [70]:
for bookname in tqdm(new_ele_list):
    update_resource_stories(['ELE', bookname])
    update_sentence_ids(['ELE', bookname])

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 27.82it/s]


------------ENG-PTF-4.2-4-Waverider------------
resource table updated :RS083
ENG-PTF-4.2-4-Waverider : story table updated  - PTF4.2
updated stories :12
------------Waverider------------
sentence table updated :953 sentences
-newly added token:xpos IDs -> 253


  9%|████                                        | 1/11 [00:48<08:09, 48.90s/it]

-newly added upos:lemma IDs -> 95
--XT ID UL ID Updated



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 46.92it/s]

...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

------------NFS-ERK-3-3-Eureka------------
resource table updated :RS093
NFS-ERK-3-3-Eureka : story table updated  - ERK3
updated stories :46
*********RS ID Duplicated : 0
*********Title Duplicated : 2


------------Eureka------------
sentence table updated :2630 sentences
-newly added token:xpos IDs -> 626


100%|███████████████████████████████████████████| 11/11 [11:23<00:00, 62.15s/it]

-newly added upos:lemma IDs -> 272
--XT ID UL ID Updated


# 1) Resource table & Story table 업데이트
#### 리스트 형식으로 ['과정', '폴더이름'] 파라미터로 입력
- Resource 테이블에 업데이트 : RS ID 알려 줌
    * 이미 있는 경우 000 is already in the reference table : RSxx 
- story 테이블에 업데이트 : 몇 개의 스토리 인지 알려 줌
    * 이미 있는 경우 story list already in the table : 폴더이름, 해당 인덱스 print 
    

In [52]:
update_resource_stories(['ELE','ENG-PTF-4.1-4-Mariner'])

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 21.51it/s]

------------ENG-PTF-4.1-4-Mariner------------
resource table updated :RS082
ENG-PTF-4.1-4-Mariner : story table updated  - PTF4.1
updated stories :12


In [53]:
with open('./data/resource_table_new.pickle', 'rb') as f:
    textbook_df = pickle.load(f)
with open('./data/story_table_text_new.pickle', 'rb') as f:
    story_df = pickle.load(f)
story_df

,RS ID,SR ID,SR Code,Group,SR Order,Unit,SR Title,Text
0,RS001,SR0001,CST04-U1-S01,G1,S01,U1,CSAT2004Q18,I have been living in this foreign country for...
1,RS001,SR0002,CST04-U1-S02,G1,S02,U1,CSAT2004Q19,"A few years ago, I was traveling in the countr..."
2,RS001,SR0003,CST04-U1-S03,G1,S03,U1,CSAT2004Q20,1. a group of similar things that one has deli...
3,RS001,SR0004,CST04-U1-S04,G1,S04,U1,CSAT2004Q21,Though we cannot dismiss Mr. Smith's opinion c...
4,RS001,SR0005,CST04-U1-S05,G1,S05,U1,CSAT2004Q22,Teens surround themselves with [BLANK] audienc...
...,...,...,...,...,...,...,...,...
2046,RS082,SR2047,PTF4.1-U2-S02,G1,S02,U2,Extreme Animals,INTRODUCTION\nThere are many kinds of environm...
2047,RS082,SR2048,PTF4.1-U2-S03,G2,S03,U2,At the Library,Rick Dodson admired the pink and orange sky as...
2048,RS082,SR2049,PTF4.1-U2-S04,G2,S04,U2,A New Bear in the Forest,Chapter 1 There’s a Bear Outside!\nJennie and ...
2049,RS082,SR2050,PTF4.1-U2-S05,G3,S05,U2,Judys Appalachia,Judy Bonds’s six-year-old grandson stood in a ...


In [50]:
story_df[story_df['SR ID'].duplicated()]

,RS ID,SR ID,SR Code,Group,SR Order,Unit,SR Title,Text


# 2) Sentence table 업데이트
#### sentence table 업데이트 + ST ID - XT ID / ST ID -UL ID + XT ID / UL ID table 모두 업데이트
#### 리스트 형식으로 ['과정', '폴더이름'] 파라미터로 입력
- sentence table의 해당 Story ID가 story table 에서 이미 구축되어 있는 경우, 메세지와 함께 중지

In [54]:
update_sentence_ids(['ELE','ENG-PTF-4.1-4-Mariner'])

------------Mariner------------
sentence table updated :887 sentences
-newly added token:xpos IDs -> 238
-newly added upos:lemma IDs -> 90
--XT ID UL ID Updated


In [55]:
with open('./data/ULID.pickle', 'rb') as f :
    lemma_upos=pickle.load(f)
lemma_upos.tail()
#int(lemma_upos.iloc[-1]['UL ID'][2:]) 
#lemma_upos[lemma_upos['UL ID'].duplicated()]

,UL ID,UPOS:Lemma
14128,UL14137,ADJ:overcrowded
14129,UL14138,NOUN:stepfather
14130,UL14139,NOUN:farmhand
14131,UL14140,NOUN:shed
14132,UL14141,ADJ:half


In [58]:
with open('./data/sentence_list_new.pickle', 'rb') as f:
    sentence_df = pickle.load(f)
with open('./data/STID_XTID_new.pickle', 'rb') as f :
    STID_XTID=pickle.load(f)
with open('./data/STID_ULID_new.pickle', 'rb') as f :
    STID_ULID=pickle.load(f)
sentence_df

,SR ID,Sentence,ST ID
0,SR0001,I have been living in this foreign country for...,ST00001
1,SR0001,Near my house is a tiny dry-cleaning shop run ...,ST00002
2,SR0001,"They are probably in their 60s, which isn't al...",ST00003
3,SR0001,Sometimes they try my patience.,ST00004
4,SR0001,Once I had to go back five times to pick up a ...,ST00005
...,...,...,...
41578,SR2051,It took 14 years to tear down the Five Points ...,ST41579
41579,SR2051,Riis’s work helped win that battle.,ST41580
41580,SR2051,Jacob Riis died in 1914.,ST41581
41581,SR2051,Jacob Riis Park in New York City is a memorial...,ST41582


# ------------------------------------------------------

# 3) 몇 개의 레퍼런스 / 스토리 / 문장에서 등장했는지 알아보기
* create_merged_df() -> 각 UL ID와 RS, SR, ST 정보 합친 테이블 생성
* get_sentence_story_textbook(st_lemma_txt)

In [9]:
st_lemma_txt=create_merged_df()
st_lemma_txt.head()

,UL ID,UPOS:Lemma,RS ID,RS Code,RS Title,SR ID,SR Code,SR Title,ST ID,Sentence
0,UL00067,VERB:live,RS001,CST-04,2004,SR0001,CST-04-S01,CSAT2004Q18,ST00001,I have been living in this foreign country for...
1,UL03396,ADJ:foreign,RS001,CST-04,2004,SR0001,CST-04-S01,CSAT2004Q18,ST00001,I have been living in this foreign country for...
2,UL00915,NOUN:country,RS001,CST-04,2004,SR0001,CST-04-S01,CSAT2004Q18,ST00001,I have been living in this foreign country for...
3,UL00182,NOUN:year,RS001,CST-04,2004,SR0001,CST-04-S01,CSAT2004Q18,ST00001,I have been living in this foreign country for...
4,UL00644,NOUN:house,RS001,CST-04,2004,SR0001,CST-04-S01,CSAT2004Q18,ST00002,Near my house is a tiny dry-cleaning shop run ...


# ------------------------------------------------------

# 4) SR & ST Count + ST/SR Count
#### parameter: ULID

In [58]:
st_lemma_txt=create_merged_df()

In [59]:
countdf=get_sr_st_count(lemma_upos2)
countdf

100%|██████████████████████████████████████| 12930/12930 [01:08<00:00, 188.51it/s]


,UL ID,UPOS:Lemma,SR Count,ST Count,ST/SR
0,UL00001,NOUN:engineer,23,40,1.74
1,UL00002,VERB:like,109,136,1.25
2,UL00003,VERB:solve,52,71,1.37
3,UL00004,NOUN:problem,180,256,1.42
4,UL00005,VERB:fix,23,29,1.26
...,...,...,...,...,...
12925,UL12934,VERB:ravage,1,1,1.00
12926,UL12935,ADJ:cyclical,1,1,1.00
12927,UL12936,VERB:converge,1,1,1.00
12928,UL12937,NOUN:upwelling,1,1,1.00


# 5) 레퍼런스 단어 제외
#### parameter: ref_df, lemma_df, condition

In [42]:
with open('./data/ULID.pickle', 'rb') as f : 
    lemma_upos=pickle.load(f)

reference=['Function Words', '교육부3K', 'Poly5K', 'CCSS3K', 'COCA6K']

with open('./data/'+reference[0]+'.pickle', 'rb') as f :
    ref_df1=pickle.load(f)
    
with open('./data/'+reference[1]+'.pickle', 'rb') as f :
    ref_df2=pickle.load(f)

with open('./data/'+reference[2]+'.pickle', 'rb') as f :
    ref_df3=pickle.load(f)

with open('./data/'+reference[3]+'.pickle', 'rb') as f :
    ref_df4=pickle.load(f)

In [14]:
ref_dec_df1=reference_words_exclusion(ref_df1,lemma_upos, 'LEMMA')
ref_dec_df2=reference_words_exclusion(ref_df2, ref_dec_df1,'LEMMA')
ref_dec_df3=reference_words_exclusion(ref_df3, ref_dec_df2,'POS:LEMMA')
ref_dec_df4=reference_words_exclusion(ref_df4, ref_dec_df3,'POS:LEMMA')

In [26]:
sr_count_df=get_sr_st_count(ref_dec_df4)

100%|████████████████████████████████████████| 5999/5999 [00:30<00:00, 197.38it/s]


# ------------------------------------------------------

# 6) UPOS:LEMMA 가 등장한 모든 문장을 테이블로
#### 파라미터는 "UPOS:LEMMA"
* RS ID, RS Code, RS Title
* SR ID, ST Title
* ST ID & Sentence

In [88]:
df=get_ul_sentence('VERB:help')
#pd.set_option('max_rows', None)
pd.set_option('max_colwidth', None)
df[['RS Code', 'Sentence']]

,RS Code,Sentence
0,CST-04,The rainforests are full of plants and animals that need each other and help each other.
1,CST-04,We need people to help with telephone calls from customers placing advertisements in our newspaper.
2,CST-04,"Because we want to help customers get the most from their advertisements, we are looking for people who can demonstrate the following:"
3,CST-04,"I learned how to play a traditional Korean musical instrument, the janggu, which helped me make friends more easily."
4,CST-04,"For example, to help busy shoppers race through the store, companies now display floor plans in many different places throughout the store."
...,...,...
626,JRN-3.1,Even the President would have to help.
627,JRN-3.1,"White House gardeners prepared the land, and elementary students from Washington, D.C., helped Mrs. Obama to plant seeds."
628,JRN-3.1,"The signal said ‘We are dying,’ and then ‘Come and help us.’”"
629,JRN-3.1,You will lead our people and help those in need.


# ------------------------------------------------------

# 7) 각 UPOS:LEMMA의 RS Code & SR Code
#### UPOS:LEMMA 가 등장하는 Resource code / Story code 를 comma 로 분리 된 테이블로

In [112]:
res_code=get_resource_code()
res_code

100%|████████████████████████████████████| 15081/15081 [01:17<00:00, 195.41it/s]


,UL ID,UPOS:Lemma,RS Code
0,UL00001,NOUN:engineer,"ISL-3, ESC-4, ESC-6, JRN3.1, ALP-V4, ANF-6, PT..."
1,UL00002,VERB:like,"MWD-2, ESC-6, MWD-3, ALP-V4, ALP-Z5, PTF3.1, E..."
2,UL00003,VERB:solve,"ESC-4, MWD-2, ESC-6, MWD-3, WDS2.2, WDS3.1, AL..."
3,UL00004,NOUN:problem,"MWD-2, ESC-6, MWD-3, ALP-V4, ANF-6, ALP-Z5, PT..."
4,UL00005,VERB:fix,"ESC-4, MWD-2, WDS2.2, MWD-3, JRN3.1, ERK3, WDS..."
...,...,...,...
15076,UL15085,NOUN:pih,ERK3
15077,UL15086,NOUN:toon,ERK3
15078,UL15087,NOUN:urn,ERK3
15079,UL15088,NOUN:482°c,ERK3


In [4]:
res_code[res_code['UPOS:Lemma']=='NOUN:engineer']['RS Code']

0    ESC-6, ISL-3, ANF-6, ESC-4, ALP-V4
Name: RS Code, dtype: object

In [5]:
res_code['RS Code'].loc[0]

'ESC-6, ISL-3, ANF-6, ESC-4, ALP-V4'

In [6]:
story_code=get_story_code()
story_code

100%|████████████████████████████████████| 13000/13000 [00:21<00:00, 594.81it/s]


,UL ID,UPOS:Lemma,SR Code
0,UL00001,NOUN:engineer,"ESC-4-S02, ESC-4-S01, ALP-V4-S09, ESC-4-S17, E..."
1,UL00002,VERB:like,"ESC-4-S20, MWD-2-S08, ISL-3-S04, ALP-Z5-S03, A..."
2,UL00003,VERB:solve,"ESC-4-S01, ESC-5-S07, ALP-Z5-S06, MWD-2-S04, E..."
3,UL00004,NOUN:problem,"ALP-P4-S08, ALP-Z5-S06, MWD-2-S04, ALP-V4-S02,..."
4,UL00005,VERB:fix,"ESC-4-S01, MWD-2-S10, MWD-1-S09, MWD-2-S02, MW..."
...,...,...,...
12995,UL13004,NOUN:disc,MWD-2-S31
12996,UL13005,ADJ:founding,MWD-2-S32
12997,UL13006,NOUN:steamboat,MWD-2-S32
12998,UL13007,NOUN:women,MWD-2-S32


In [7]:
story_code['SR Code'].loc[0]

'ESC-4-S02, ESC-4-S01, ALP-V4-S09, ESC-4-S17, ESC-6-S17, ISL-3-S06, ANF-6-S01, ANF-6-S02, ANF-6-S08, ANF-6-S12'